# WanDBのSweepを使ってみる

### wandbへのログイン
APIキーを使います．

In [ ]:
import wandb
wandb.login()


### 訓練の定義
wandb sweepでは，sweepで訓練対象の関数を指定する必要があります．
sweep内でwandb.configを用いることでハイパラを取得できます．
クラスを呼び出す場合，クラスを呼び出す関数をsweepする感じで実行します．

In [ ]:
# 1: Define objective/training function
def objective(config):
    score = config.x**3 + config.y
    return score

In [ ]:
def sweeper():
    wandb.init(project="Test")
    score = objective(wandb.config)
    wandb.log({"score": score})

### ハイパラのconfig読み込み
wandbのsweepでは，ハイパラをyamlで保存できます．
python上でも定義できますが，yaml管理の方が楽です．
具体的なパラメータ設定，optunaの適用方法はsweep_config.mdに書いときます．

In [ ]:
# 2: Define the search space
sweep_config = {
    "method": "random",
    "metric": {"goal": "minimize", "name": "score"},
    "parameters": {
        "x": {"max": 0.1, "min": 0.01},
        "y": {"values": [1, 3, 7]},
    },
}

In [ ]:
# %pip install pyyaml

In [ ]:
import yaml

with open("config.yaml", "r") as file:
    sweep_config = yaml.safe_load(file)

### sweepの実行
sweep名を定義して，agentを動かせば実行できます．

In [ ]:
# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="test-sweep")

wandb.agent(sweep_id, function=sweeper, count=10)